In [ ]:
cd ../../UNIT

In [2]:
#from trainer import UNIT_Trainer
#import xarray as xr
from trainer import UNIT_Trainer
from utils import get_all_data_loaders, prepare_sub_folder, write_html, write_loss, get_config, write_2images, Timer

import argparse
import os
import sys
import shutil

import torch
from torch.autograd import Variable
import torch.backends.cudnn as cudnn


In [3]:
torch.__version__

'1.6.0'

In [4]:
config_path = 'configs/hadgem3_to_cam5_nat-hist-4channels.yaml'
cudnn.benchmark = True

# Load experiment setting
config = get_config(config_path)
config['num_workers']=3
config['timeout']=30
max_iter = config['max_iter']
display_size = 2

In [5]:
train_loader_a, _, train_loader_b, _ = get_all_data_loaders(config, downscale_consolidate=False)

In [6]:
# Start training
for it, (images_a, images_b) in enumerate(zip(train_loader_a, train_loader_b)):
    images_a.cuda()
    print(it)
    if it >=5:
        break

0
1
2
3
4
5


In [7]:
images_a.shape

torch.Size([1, 4, 324, 432])

In [8]:
# Selection of climate fields to display after a number of updates
train_display_images_a = torch.cat([img for _, img in zip(range(display_size), train_loader_a)]).cuda()
train_display_images_b = torch.cat([img for _, img in zip(range(display_size), train_loader_b)]).cuda()

In [9]:
train_display_images_a

tensor([[[[-0.5515, -0.5263, -0.5288,  ..., -0.5508, -0.5489, -0.5763],
          [-0.5240, -0.5252, -0.5240,  ..., -0.5172, -0.5188, -0.5152],
          [-0.5118, -0.5130, -0.5142,  ..., -0.5051, -0.5069, -0.5082],
          ...,
          [-0.3149, -0.3150, -0.3151,  ..., -0.3158, -0.3158, -0.3153],
          [-0.3268, -0.3267, -0.3266,  ..., -0.3267, -0.3267, -0.3266],
          [-0.4487, -0.4489, -0.4492,  ..., -0.4479, -0.4482, -0.4486]],

         [[ 1.6043,  1.5984,  1.5955,  ...,  1.6027,  1.6029,  1.6105],
          [ 1.5843,  1.5876,  1.5898,  ...,  1.5948,  1.5900,  1.5848],
          [ 1.6405,  1.6392,  1.6393,  ...,  1.6450,  1.6431,  1.6401],
          ...,
          [-0.9577, -0.9586, -0.9561,  ..., -0.9604, -0.9585, -0.9619],
          [-0.9656, -0.9646, -0.9621,  ..., -0.9671, -0.9657, -0.9679],
          [-1.0111, -1.0088, -1.0080,  ..., -1.0087, -1.0109, -1.0102]],

         [[ 1.6818,  1.6758,  1.6723,  ...,  1.6728,  1.6755,  1.6852],
          [ 1.6535,  1.6581,  

In [19]:
import xarray as xr

ds = xr.load_dataset(config['agg_data_a'])
ds['tasmin'].values = ds['tas'].values
ds['tasmax'].values = ds['tas'].values
ds.to_netcdf(config['agg_data_a'].strip('.nc')+'_temp_coherent.nc')

In [20]:
import xarray as xr

ds = xr.load_dataset(config['agg_data_b'])
ds['tasmin'].values = ds['tas'].values
ds['tasmax'].values = ds['tas'].values
ds.to_netcdf(config['agg_data_b'].strip('.nc')+'_temp_coherent.nc')

In [22]:
f = '/datadrive/miroc5/nat_hist_agg.nc'
f

'/datadrive/miroc5/nat_hist_agg.nc'

In [23]:
ds = xr.load_dataset(f)
ds['tasmin'].values = ds['tas'].values
ds['tasmax'].values = ds['tas'].values
ds.to_netcdf(f.strip('.nc')+'_temp_coherent.nc')